In [1]:
import pandas as pd
import os
import re
from soynlp.hangle import jamo_levenshtein
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook

In [2]:
def split_name(name):
    hangul = re.match(r'^[가-힣]{2,5}', name)
    number = re.search(r'\d{4}$', name)
    if hangul and number:
        return hangul.group(), number.group()
    return name, ''

In [3]:
def find_similar_names(target_hangul, names, threshold=0.5):
    similar_names = []
    for name in names:
        name_hangul, _ = split_name(name)
        similarity = 1 - jamo_levenshtein(target_hangul, name_hangul)
        if similarity > threshold:
            similar_names.append((name, similarity))
    return sorted(similar_names, key=lambda x: x[1], reverse=True)

In [4]:
def process_outliers(outlier, rb):
    ol = {}
    for k, v in outlier.items():
        print(f"{k} 처리 중...")
        for outlier_name in v:
            outlier_hangul, outlier_number = split_name(outlier_name)
            
            for sheet_name, sheet_df in rb.items():
                names = sheet_df['이름+전화번호 뒤 4자리'].tolist()
                similar_names = find_similar_names(outlier_hangul, names)
                
                for name, similarity in similar_names:
                    _, name_number = split_name(name)
                    if outlier_number == name_number:
                        print(f"  {outlier_name} -> {sheet_name} {name} (유사도: {similarity:.2f})")
                        ol[f'{outlier_name}'] = f'{sheet_name} - {name}'
                        break
                else:
                    continue
                break
            else:
                print(f"  {outlier_name}에 대한 유사한 이름을 찾을 수 없습니다.")

In [5]:
def save_results(rb, filename):
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for sheet_name, df in rb.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"결과가 {filename}에 저장되었습니다.")

def load_results(filename):
    return pd.read_excel(filename, sheet_name=None)

In [6]:
def process_attendance(rb, attendance_df, round_num):
    outlier = {}
    attendance_column = f'{round_num}차 출결'

    for _, row in attendance_df.iterrows():
        id = row[attendance_df.columns[2]]
        dept = row[attendance_df.columns[3]].replace('수도권 ', '')
        
        found = False
        for sheet_name, sheet_df in rb.items():
            mask = (sheet_df['이름+전화번호 뒤 4자리'] == id)
            if mask.any():
                rb[sheet_name].loc[mask, attendance_column] = 1
                found = True
                break
        
        if not found:
            if attendance_column not in outlier:
                outlier[attendance_column] = []
            outlier[attendance_column].append(id)
            print(f"{attendance_column}: ID {id}에 해당하는 사용자를 찾을 수 없습니다.")

    # NaN 값을 0으로 채우기
    for k in rb.keys():
        rb[k][attendance_column] = rb[k][attendance_column].fillna(0)

    # outlier 처리
    process_outliers(outlier, rb)

def apply_excel_formulas(filename):
    wb = load_workbook(filename)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        max_row = ws.max_row
        max_col = ws.max_column
        
        # 출결 열 찾기
        attendance_cols = []
        for col in range(1, max_col + 1):
            cell_value = ws.cell(row=1, column=col).value
            if isinstance(cell_value, str) and '차 출결' in cell_value:
                attendance_cols.append(col)
        
        if not attendance_cols:
            print(f"'{sheet}' 시트에서 출결 열을 찾을 수 없습니다.")
            continue
        
        # '출결 횟수' 열 추가 및 함수식 적용
        attendance_count_col = max_col + 1
        ws.cell(row=1, column=attendance_count_col, value='출결 횟수')
        for row in range(2, max_row + 1):
            formula = f'=SUM({",".join([f"{get_column_letter(col)}{row}" for col in attendance_cols])})'
            ws.cell(row=row, column=attendance_count_col, value=formula)
        
        # '최종 출결' 열 추가 및 함수식 적용
        final_attendance_col = max_col + 2
        ws.cell(row=1, column=final_attendance_col, value='최종 출결')
        attendance_count_col_letter = get_column_letter(attendance_count_col)
        total_sessions = len(attendance_cols)
        
        for row in range(2, max_row + 1):
            conditions = []
            conditions.append(f'{attendance_count_col_letter}{row}>={total_sessions-1}')  # 전체 회차의 1회 미만 결석
            
            # 1, 2차 연속 결석이나 마지막 두 차 연속 결석 시 5회 출석이면 출석으로 인정
            if total_sessions >= 2:
                first_two = f'AND({attendance_count_col_letter}{row}={total_sessions-2},{get_column_letter(attendance_cols[0])}{row}=0,{get_column_letter(attendance_cols[1])}{row}=0)'
                last_two = f'AND({attendance_count_col_letter}{row}={total_sessions-2},{get_column_letter(attendance_cols[-2])}{row}=0,{get_column_letter(attendance_cols[-1])}{row}=0)'
                conditions.append(first_two)
                conditions.append(last_two)
            
            formula = f'=IF(OR({",".join(conditions)}),"출결","결석")'
            ws.cell(row=row, column=final_attendance_col, value=formula)
    
    wb.save(filename)
    print(f"Excel 함수식이 {filename}에 적용되었습니다.")

In [7]:
# 사용자 정보 파일 읽기
rb = pd.read_excel('user_info/users.xlsx',
                    sheet_name=['1분과', '2분과', '3분과', '4분과', '충청분과', '영남분과'])

# 출결 파일 점진적 처리
for i in range(1, 8):
    # 이전 회차의 처리 결과 확인
    prev_result_file = f'processed_attendance_{i-1}차.xlsx'
    if i > 1 and os.path.exists(prev_result_file):
        print(f"{i-1}차 처리 결과를 불러옵니다.")
        rb = load_results(prev_result_file)

    current_path = f'./check/{i}차 출결.xlsx'
    if os.path.exists(current_path):
        print(f"{i}차 출결 파일 처리 중...")
        attendance_df = pd.read_excel(current_path)
        process_attendance(rb, attendance_df, i)
        
        # 중간 결과 저장
        save_results(rb, f'processed_attendance_{i}차.xlsx')
    else:
        print(f"{i}차 출결 파일이 존재하지 않습니다. 처리를 종료합니다.")
        break

1차 출결 파일 처리 중...
1차 출결: ID 정지욘2320에 해당하는 사용자를 찾을 수 없습니다.
1차 출결: ID 박예린6826에 해당하는 사용자를 찾을 수 없습니다.
1차 출결 처리 중...
  정지욘2320 -> 2분과 정지용2320 (유사도: 0.67)
  박예린6826에 대한 유사한 이름을 찾을 수 없습니다.
결과가 processed_attendance_1차.xlsx에 저장되었습니다.
1차 처리 결과를 불러옵니다.
2차 출결 파일 처리 중...
2차 출결: ID 장지용2320에 해당하는 사용자를 찾을 수 없습니다.
2차 출결: ID 이수정4955에 해당하는 사용자를 찾을 수 없습니다.
2차 출결: ID 박예린6826에 해당하는 사용자를 찾을 수 없습니다.
2차 출결 처리 중...
  장지용2320 -> 2분과 정지용2320 (유사도: 0.67)
  이수정4955에 대한 유사한 이름을 찾을 수 없습니다.
  박예린6826에 대한 유사한 이름을 찾을 수 없습니다.
결과가 processed_attendance_2차.xlsx에 저장되었습니다.
2차 처리 결과를 불러옵니다.
3차 출결 파일 처리 중...
3차 출결: ID 김효리9768에 해당하는 사용자를 찾을 수 없습니다.
3차 출결: ID 김민채9204에 해당하는 사용자를 찾을 수 없습니다.
3차 출결: ID 차신영9314에 해당하는 사용자를 찾을 수 없습니다.
3차 출결 처리 중...
  김효리9768에 대한 유사한 이름을 찾을 수 없습니다.
  김민채9204에 대한 유사한 이름을 찾을 수 없습니다.
  차신영9314에 대한 유사한 이름을 찾을 수 없습니다.
결과가 processed_attendance_3차.xlsx에 저장되었습니다.
3차 처리 결과를 불러옵니다.
4차 출결 파일 처리 중...
4차 출결: ID 강운솔6341에 해당하는 사용자를 찾을 수 없습니다.
4차 출결: ID 김태윤3388에 해당하는 사용자를 찾을 수 없습니다.
4차 출결 처리 중...
  강운솔6341 -> 3분과 강은솔6341